In [1]:
%load_ext autoreload
%autoreload 2
import sys
import spacy
import pandas as pd
from citepred import utils, llm
import logging
logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
paths = utils.get_paths()
paths

Paths(raw_data=PosixPath('/Users/grabski/publication-citation-prediction/data/raw'))

In [4]:
publications = pd.read_parquet(paths.raw_data / 'publications.parquet')
publications.info()
publications.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199118 entries, 0 to 199117
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   pmid               199118 non-null  int64  
 1   pub_year           199118 non-null  int64  
 2   journal_iso        199118 non-null  object 
 3   journal_citescore  199118 non-null  float64
 4   abstract           199118 non-null  object 
 5   title              199118 non-null  object 
 6   authors            199118 non-null  object 
 7   affiliations       199118 non-null  object 
 8   mesh_terms         199118 non-null  object 
 9   references         199118 non-null  object 
 10  citations          199118 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 16.7+ MB


,pmid,pub_year,journal_iso,journal_citescore,abstract,title,authors,affiliations,mesh_terms,references,citations
0,3319190,1987,Cell,63.400002,The protein product of the human Duchenne musc...,Dystrophin: the protein product of the Duchenn...,"[{'author_pos': 2, 'last_name': 'Brown', 'fore...","[{'author_pos': 1, 'affiliation_pos': 1, 'affi...","[{'descriptor_ui': 'D004195', 'descriptor_name...","[6313869, 3888377, 3711930, 6330056, 3962599, ...","[{'pmid': 27924830, 'pub_year': 2016}, {'pmid'..."
1,2446136,1987,Nature,56.900002,"Organic substrates (sugars, amino acids, carbo...",Expression cloning and cDNA sequencing of the ...,"[{'author_pos': 3, 'last_name': 'Ikeda', 'fore...","[{'author_pos': 1, 'affiliation_pos': 1, 'affi...","[{'descriptor_ui': 'D014982', 'descriptor_name...","[2876429, 6444453, 6198242, 3472228, 3027132, ...","[{'pmid': 25989422, 'pub_year': 2015}, {'pmid'..."
2,3183348,1988,J Hepatol,33.599998,In order to understand the pathophysiology of ...,Etiologic factors of jaundice in severely ill ...,"[{'author_pos': 4, 'last_name': 'Yap', 'fore_n...","[{'author_pos': 1, 'affiliation_pos': 1, 'affi...","[{'descriptor_ui': 'D008297', 'descriptor_name...","[472994, 3966830, 6754962, 6837052, 6849639, 1...","[{'pmid': 36289602, 'pub_year': 2022}, {'pmid'..."
3,8479518,1993,Nature,56.900002,"Atherosclerosis, the principal cause of heart ...",The pathogenesis of atherosclerosis: a perspec...,"[{'author_pos': 1, 'last_name': 'Ross', 'fore_...","[{'author_pos': 1, 'affiliation_pos': 1, 'affi...","[{'descriptor_ui': 'D006801', 'descriptor_name...","[1699098, 1990208, 2451132, 218198, 2196221, 2...","[{'pmid': 16339764, 'pub_year': 2005}, {'pmid'..."
4,8309034,1994,JAMA,24.799999,"We recorded study design, age range of study p...",Efficacy of BCG vaccine in the prevention of t...,"[{'author_pos': 4, 'last_name': 'Wilson', 'for...","[{'author_pos': 1, 'affiliation_pos': 1, 'affi...","[{'descriptor_ui': 'D006801', 'descriptor_name...","[1518584, 13519929, 8381207, 4811772, 3209343,...","[{'pmid': 18024541, 'pub_year': 2007}, {'pmid'..."


In [7]:
sample_abstracts = publications.sample(n=3, random_state=0)['abstract']
sample_sentences = sample_abstracts.apply(lambda text: [str(e) for e in nlp(text).sents]).explode()
print("\n".join(sample_sentences.tolist()))

We included 22 trials comprising 1216 patients.
Dehydrated amniotic membrane injections were significantly superior to corticosteroids in the short term in achieving the primary and composite outcomes (mean difference (MD) in visual analogue scale (VAS) was -7.32, 95% CIs -11.2 to -3.38; and MD in the foot health status questionnaire was 31.2, 95% CIs 13.9 to 48.6, respectively).
For pain relief, botulinum toxin-A provided a significant short-term advantage over placebo, which was still present at 6 months (MD in VAS was -2.9, 95% CIs -4.44 to -1.39; and MD -4.34, 95% CIs -7.18 to -1.54, respectively).
Insect color patterns can be very diverse.
This variation is also seen among many larval instar stages, which can take on vastly different phenotypes.
Young caterpillars of the swallowtail butterfly, Papilio xuthus, are mimics of bird droppings, whereas the fifth larval instar is camouflaged among the leaves of host plants (cryptic pattern).
We find that juvenile hormone (JH) titers decr

In [16]:
sample_innovation = llm.get_innovation(sample_sentences)
sample_innovation

DEBUG:citepred.llm:sentences:
['We included 22 trials comprising 1216 patients.', 'Dehydrated amniotic membrane injections were significantly superior to corticosteroids in the short term in achieving the primary and composite outcomes (mean difference (MD) in visual analogue scale (VAS) was -7.32, 95% CIs -11.2 to -3.38; and MD in the foot health status questionnaire was 31.2, 95% CIs 13.9 to 48.6, respectively).']
DEBUG:citepred.llm:Clean Sentences:
['We included 22 trials comprising 1216 patients.', 'Dehydrated amniotic membrane injections were significantly superior to corticosteroids in the short term in achieving the primary and composite outcomes (mean difference (MD) in visual analogue scale (VAS) was -7.32, 95% CIs -11.2 to -3.38; and MD in the foot health status questionnaire was 31.2, 95% CIs 13.9 to 48.6, respectively).']
DEBUG:citepred.llm:Prompt:
Classify the sentiment of the following sentences from published scientific articles as either "innovative", "non-innovative", 

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
with pd.option_context('display.max_colwidth', 250):
    display(
        pd.DataFrame(dict(score=sample_scores, sentence=sample_sentences))
        .reset_index(drop=True)
    )

,score,sentence
0,77.0,We included 22 trials comprising 1216 patients.
1,15.0,"Dehydrated amniotic membrane injections were significantly superior to corticosteroids in the short term in achieving the primary and composite outcomes (mean difference (MD) in visual analogue scale (VAS) was -7.32, 95% CIs -11.2 to -3.38; and M..."
2,17.0,"For pain relief, botulinum toxin-A provided a significant short-term advantage over placebo, which was still present at 6 months (MD in VAS was -2.9, 95% CIs -4.44 to -1.39; and MD -4.34, 95% CIs -7.18 to -1.54, respectively)."
3,80.0,Insect color patterns can be very diverse.
4,60.0,"This variation is also seen among many larval instar stages, which can take on vastly different phenotypes."
5,25.0,"Young caterpillars of the swallowtail butterfly, Papilio xuthus, are mimics of bird droppings, whereas the fifth larval instar is camouflaged among the leaves of host plants (cryptic pattern)."
6,47.0,We find that juvenile hormone (JH) titers decrease during the fourth larval instar.
7,16.0,"Furthermore, treatment with JH analog at the beginning of the fourth instar stage resulted in reproducing the mimetic pattern instead of the usual cryptic one and likewise altered gene expression patterns to that associated with the mimetic pattern."
8,43.0,These findings suggest that JH regulates the progressive larval pattern switch of this insect.
9,NaN,"To integrate available clinical and molecular information for cytogenetically normal acute myeloid leukemia (CN-AML) patients into one risk score, 275 CN-AML patients from multicenter treatment trials AML SHG Hannover 0199 and 0295 and 131 patien..."
